In [4]:
import os, treetaggerwrapper, networkx as nx, collections
import pandas as pd
import numpy as np
from HanTa import HanoverTagger as ht
import nltk, string
from nltk.stem.snowball import GermanStemmer
from nltk.stem import PorterStemmer
from nltk import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics, neighbors
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from pprint import pprint
nltk.download('wordnet')
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from scipy import sparse

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dorian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('G:/Extractive-Summarisation-of-German-Wikipedia/dataset/data_class.csv', encoding='utf-8')

In [5]:
nltk.download('wordnet') # first-time use only
german_stop_words = stopwords.words('german')
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dorian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
stopword_list = nltk.corpus.stopwords.words('german')

tv = TfidfVectorizer(min_df = 0.05, max_df = 0.8, tokenizer=LemNormalize, stop_words = stopword_list)
X = tv.fit_transform(df['source1'])
vocab = tv.get_feature_names()

H:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['al', 'au', 'bi', 'de', 'diesis', 'dy', 'e', 'mus', 'un', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [7]:
X1 = pd.DataFrame(X.toarray(), columns = vocab)
X1['no_words_inSent_SS	'] = df['no_words_inSent_SS']
X1['no_words_inSent_SK'] = df['no_words_inSent_SK']
X1['noCap_LetterWords_inSentence'] = df['noCap_LetterWords_inSentence']

In [8]:
X_sparse = sparse.csr_matrix(X1.values)

In [9]:
print(X_sparse.shape)

(3513935, 5)


In [10]:
X = X_sparse.toarray()
y = df.sim_sent.values
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3)

In [11]:
clf = MultinomialNB()
clf.fit(X_train,y_train)

MultinomialNB()

In [12]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
predict = clf.predict(X_test)
print("\n============ Naive Bayes")
print("---- confusion_matrix:")
print(confusion_matrix(y_test,predict))


============ Naive Bayes
---- confusion_matrix:
[[     4 515678]
 [     2 538497]]


In [13]:
print("---- classification_report:")
print(classification_report(y_test,predict))
print("---- accuracy_score:")
print(accuracy_score(y_test,predict))

---- classification_report:
              precision    recall  f1-score   support

           0       0.67      0.00      0.00    515682
           1       0.51      1.00      0.68    538499

    accuracy                           0.51   1054181
   macro avg       0.59      0.50      0.34   1054181
weighted avg       0.59      0.51      0.35   1054181

---- accuracy_score:
0.5108240425505677


In [14]:
from sklearn.naive_bayes import GaussianNB
gnb_classifier = GaussianNB()
gnb_classifier.fit(X_train, y_train)
y_pred = gnb_classifier.predict(X_test)
print("\n============ Naive Bayes")
print("---- confusion_matrix:")
print(confusion_matrix(y_test,y_pred))
print("---- classification_report:")
print(classification_report(y_test,y_pred))
print("---- accuracy_score:")
print(accuracy_score(y_test,y_pred))


============ Naive Bayes
---- confusion_matrix:
[[ 15474 500208]
 [ 16450 522049]]
---- classification_report:
              precision    recall  f1-score   support

           0       0.48      0.03      0.06    515682
           1       0.51      0.97      0.67    538499

    accuracy                           0.51   1054181
   macro avg       0.50      0.50      0.36   1054181
weighted avg       0.50      0.51      0.37   1054181

---- accuracy_score:
0.5098963081292491


In [15]:
from sklearn.ensemble import RandomForestClassifier

forest_classifier = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
forest_classifier.fit(X_train, y_train)

y_pred = forest_classifier.predict(X_test)
print("\n============ Random Forest")
print("---- confusion_matrix:")
print(confusion_matrix(y_test,y_pred))
print("---- classification_report:")
print(classification_report(y_test,y_pred))
print("---- accuracy_score:")
print(accuracy_score(y_test,y_pred))


============ Random Forest
---- confusion_matrix:
[[     3 515679]
 [     2 538497]]
---- classification_report:
              precision    recall  f1-score   support

           0       0.60      0.00      0.00    515682
           1       0.51      1.00      0.68    538499

    accuracy                           0.51   1054181
   macro avg       0.56      0.50      0.34   1054181
weighted avg       0.55      0.51      0.35   1054181

---- accuracy_score:
0.5108230939468649


In [ ]:
from sklearn.svm import SVC

svm_classifier = SVC(gamma='scale')
svm_classifier.fit(X_train, y_train)

y_pred = svm_classifier.predict(X_test)
print("\n============ Support Vector Machine")
print("---- confusion_matrix:")
print(confusion_matrix(y_test,y_pred))
print("---- classification_report:")
print(classification_report(y_test,y_pred))
print("---- accuracy_score:")
print(accuracy_score(y_test,y_pred))

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
plt.title("Distribution in Summary important sentences Class Labels")
plt.bar(dict(Counter(df['sim_sent'])).keys(), dict(Counter(df['sim_sent'])).values())